## Issues
- jessie release only found in https://qa.debian.org/madison.php?package=cmake, not found in http://ftp.debian.org/debian/pool/main/c/cmake/ 
- slow speed of getting sources? is there a faster way?
- how to deal with new versions without date? (so far only 1624 debian library versions without release date in scantist_library_version)
- link to data-staging-test instead of sqlite for better monitoring, and data persistence

## All functions

In [225]:
## DB functions
import os
import psycopg2
import sqlite3
import pandas as pd
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from datetime import datetime
load_dotenv('../pipeline_monitoring/.env')

def sqlite_create_conn(db_filename):
    '''
    Create connection to temp sqlite database
    '''
    conn = sqlite3.connect(db_filename)
    return conn

def sqlite_init_tables(conn):
    '''
    Create scantist_library and scantist_library_version tables in sqlite
    '''
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS scantist_library (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        created timestamptz NOT NULL,
        modified timestamptz NOT NULL,
        "name" varchar(255) NULL,
        description text NULL,
        vendor varchar(255) NULL,
        "language" varchar(64) NULL,
        platform varchar(255) NULL,
        "source" varchar(255) NULL,
        processed_time timestamptz NULL,
        is_valid bool NULL,
        CONSTRAINT scantist_library_name_vendor_platform_source_db9dacce_uniq UNIQUE (name, vendor, platform)
    );''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS scantist_library_version (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        created timestamptz NOT NULL,
        modified timestamptz NOT NULL,
        version_number varchar(255) NULL,
        release_date timestamptz NULL,
        is_active bool NOT NULL,
        is_officially_supported bool NOT NULL,
        library_id int4 NULL,
        license_id int4 NULL,
        is_clean bool NOT NULL,
        "source" varchar(16) NULL,
        processed_time timestamptz NULL,
        is_valid bool NULL,
        CONSTRAINT scantist_library_version_unique_key UNIQUE (library_id, version_number)
    );''')


    conn.commit()
    cursor.close()
    conn.close()


def cve_create_conn():
    '''
    Creation connection to CVETRIAGE database
    '''
    # Replace these with your own database credentials
    host2 = os.environ.get('CVETRIAGE_POSTGRES_HOSTNAME')
    database2 = os.environ.get('CVETRIAGE_POSTGRES_DB')
    user2 = os.environ.get('CVETRIAGE_POSTGRES_USERNAME')
    password2 = os.environ.get('CVETRIAGE_POSTGRES_PASSWORD')

    # Establish the connection
    connection_cve = psycopg2.connect(
        host=host2,
        database=database2,
        user=user2,
        password=password2
    )
    return connection_cve

def querysqlite_insert_library_multiple(conn,debian_binaries_to_crawl,ftp_dict):
    '''
    Upsert into scantist_library
    debian_binaries_to_crawl:
    ftp_dict:
    '''
    cursor = conn.cursor()
    debian_binary_tup=[]
    curr_time=datetime.now().strftime("%Y-%m-%d")
    for binary_name in debian_binaries_to_crawl.keys():
        # for v in debian_binaries_to_crawl[binary_name]:
        #     if v in ftp_dict[binary_name].keys():
        debian_binary_tup.append((curr_time,curr_time,binary_name,'','','','Debian','Scantist',None,True))
    columns_library=', '.join(['created','modified','name','description','vendor','language','platform','source','processed_time','is_valid'])
    sql_query_insert_library = f"INSERT OR IGNORE INTO scantist_library ({columns_library}) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.executemany(sql_query_insert_library, debian_binary_tup)
    # Commit the changes to save them to the database file
    conn.commit()
    cursor.close()

def querysqlite_insert_library_versions_ea_binary(conn,binary_name,debian_binaries_to_crawl,ftp_dict):
    '''

    '''
    cursor = conn.cursor()
    curr_time=datetime.now().strftime("%Y-%m-%d")
    sql_query_select_library_id=f"select id from scantist_library where name='{binary_name}'"
    columns_library_version=', '.join(['created','modified','version_number','release_date','is_active','is_officially_supported','library_id','license_id','is_clean','source','processed_time','is_valid'])
    sql_query_insert_library_version = f"INSERT OR IGNORE INTO scantist_library_version ({columns_library_version}) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    
    # get library_id of binary
    cursor.execute(sql_query_select_library_id)
    result_id = cursor.fetchall()[0][0]
    print(f'Library_id for {binary_name} is {result_id}')

    debian_binary_v_tup=[]
    for v in debian_binaries_to_crawl[binary_name]:
        if v in ftp_dict[binary_name].keys():
            debian_binary_v_tup.append((curr_time,curr_time,v,min(ftp_dict[binary_name][v]),True,True,result_id,None,True,'Scantist',None,True))

    cursor.executemany(sql_query_insert_library_version, debian_binary_v_tup)
    # Commit the changes to save them to the database file
    conn.commit()
    cursor.close()

def querysqlite_insert_library_versions_ea_binary2(conn,binary_name,version_date_list):
    '''

    '''
    cursor = conn.cursor()
    curr_time=datetime.now().strftime("%Y-%m-%d")
    sql_query_select_library_id=f"select id from scantist_library where name='{binary_name}'"
    columns_library_version=', '.join(['created','modified','version_number','release_date','is_active','is_officially_supported','library_id','license_id','is_clean','source','processed_time','is_valid'])
    sql_query_insert_library_version = f"INSERT OR IGNORE INTO scantist_library_version ({columns_library_version}) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    
    # get library_id of binary
    cursor.execute(sql_query_select_library_id)
    result_id = cursor.fetchall()[0][0]
    print(f'Library_id for {binary_name} is {result_id}')

    debian_binary_v_tup=[]
    for v_date in version_date_list:
        debian_binary_v_tup.append((curr_time,curr_time,v_date[0],v_date[1],True,True,result_id,None,True,'Scantist',None,True))

    cursor.executemany(sql_query_insert_library_version, debian_binary_v_tup)
    # Commit the changes to save them to the database file
    conn.commit()
    cursor.close()

def udd_create_conn():
    '''
    Create connection to UDD database
    '''
    host1 = "udd-mirror.debian.net"
    database1 = "udd"
    user1 = "udd-mirror"
    password1 = "udd-mirror"

    # Establish the connection
    connection_udd = psycopg2.connect(
        host=host1,
        database=database1,
        user=user1,
        password=password1
    )
    return connection_udd

def querycve_all_debian_binary(connection_cve):
    '''
    
    '''
    cursor = connection_cve.cursor()
    query_debian_select_binaries = f'''SELECT name, description
    from scantist_library
    where platform='Debian'
    and is_valid=True
    '''
    cursor.execute(query_debian_select_binaries)
    result = cursor.fetchall()
    # df_debian_binaries = pd.read_sql_query(query_debian_select_binaries, connection_cve)
    # debian_binary_list = df_debian_binaries.name.to_list()
    return result

def querycve_binary_version_date_mapping(connection_cve,debian_binary_list):
    '''
    
    '''
    binary_version_mapping = {}
    cursor = connection_cve.cursor()
    for binary_name in debian_binary_list:
        # Execute your SQL query
        query_binary_versions = f'''SELECT slv.version_number, slv.release_date from scantist_library sl
        join scantist_library_version slv on slv.library_id=sl.id
        where sl.name='{binary_name[0]}'
        and sl.platform='Debian'
        and slv.is_valid=True
        order by slv.release_date desc
        '''
        cursor.execute(query_binary_versions)

        # Fetch all the rows as a list of tuples
        result = cursor.fetchall()
        binary_version_mapping[binary_name] = result
    cursor.close()
    return binary_version_mapping

def queryudd_cvetriage_missing_binary_versions(connection_udd,binary_version_mapping):
    '''

    '''
    cursor = connection_udd.cursor()
    debian_binaries_to_crawl = {}
    for binary_info in binary_version_mapping.keys():
        # Execute your SQL query
        binary_name=binary_info[0]
        query_latest_binary_versions = f"SELECT distinct version FROM public.packages where package='{binary_name}'"
        cursor.execute(query_latest_binary_versions)
        # Fetch all the rows as a list of tuples
        udd_latest_versions_tup_list = cursor.fetchall()
        udd_latest_versions_set = set([tup[0] for tup in udd_latest_versions_tup_list])
        cve_versions_set = set(tup[0] for tup in binary_version_mapping[binary_info])
        missing_latest_versions_set = udd_latest_versions_set - cve_versions_set
        debian_binaries_to_crawl[binary_name]=missing_latest_versions_set
    cursor.close()
    return debian_binaries_to_crawl

def queryudd_source_all_version_date_df(connection_udd,source_name):
    '''
    
    '''
    query = f"SELECT version, date FROM upload_history where source='{source_name}'"
    df = pd.read_sql_query(query, connection_udd)
    return df

## webscrape
def web_parse_ftp_binary_pool_latest_versions(url):
    '''
    For a given source (based on url provided), 
    get the respective binaries latest version info (binary file_name: <binary-name>_<version>_<architecture>.deb and last_modified)
    '''
    file_info_list = []
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the webpage using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <tr> elements on the webpage
        tr_elements = soup.find_all('tr')

        # Extract the names and last modified timestamps from each <tr> element
        
        for tr in tr_elements:
            td_elements = tr.find_all('td')
            if len(td_elements) >= 4:
                name = td_elements[1].text.strip()
                last_modified = td_elements[2].text.strip()
                file_info_list.append((name, last_modified))
        file_info_list.remove(('Parent Directory', ''))
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    
    return file_info_list

def web_parse_ftp_binary_pool_latest_versions_map(ftp_list):
    '''
    From the ftp_list (binary filename & last_modified),
    group by binary_name, version, and last_modified
    '''
    ftp_dict={}
    for _ in ftp_list:
        if _[0].endswith('.deb'): # filter out non deb files
            binary_name=_[0].split('_')[0]
            binary_version=_[0].split('_')[1]
            
            release_date=_[1].split(' ')[0]
            if binary_name not in ftp_dict:
                ftp_dict[binary_name]={}
            if binary_version in ftp_dict[binary_name]:
                ftp_dict[binary_name][binary_version].update([release_date])
            else:
                ftp_dict[binary_name][binary_version]=set([release_date])
    return ftp_dict

def web_binary_all_versions_df(binary_name):
    url = f'https://snapshot.debian.org/binary/{binary_name}/'
    response = requests.get(url)

    # Initialize an empty list to store the extracted text
    text_list = []

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all the <ul> elements
        ul_elements = soup.find_all('ul')

        # Loop through the <ul> elements to find <li> and <a> elements
        for ul in ul_elements:
            # Find all the <li> elements within the current <ul>
            li_elements = ul.find_all('li')
            
            # Loop through the <li> elements to find <a> elements
            for li in li_elements:
                # Find all the <a> elements within the current <li>
                a_elements = li.find_all('a')
                
                # Loop through the <a> elements to extract and append their text to the list
                for a in a_elements:
                    text = a.get_text()
                    text_list.append(text)
    binary_pkg_v_map={}
    for text in text_list:
        # Define a regular expression pattern to match 'XXX' and 'ZZZ@WWW'
        # pattern = r'/(\b[^()]+\b)\s\(source:\s*[^()]+\s([^()]+)\)/gm'
        pattern = r'(\S+)\s\(source:\s*[^()]+\s(\S+)\)'

        # Use re.search to find the pattern in the text
        match = re.search(pattern, text)

        # Check if a match is found
        if match:
            binary_v = match.group(1)  
            pkg_v = match.group(2)  
            binary_pkg_v_map[binary_v]=pkg_v

    df1 = pd.DataFrame(binary_pkg_v_map.items(), columns=['binary_v', 'source_v'])
    return df1

def get_binary_source(binary_name):
    '''

    '''
    # Send an HTTP GET request to the URL
    url = f'https://tracker.debian.org/pkg/{binary_name}'
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the <span> element with the specific class
        span_element = soup.find('span', class_='list-item-key')

        # Check if the <span> element was found
        if span_element:
            # Find the following <a> element
            a_element = span_element.find_next('a')
            
            # Extract and print the text within the <a> element
            if a_element:
                href_text = a_element.get_text()
            else:
                print("No <a> element found following the <span>.")
        else:
            print("No <span> element with the specified class found.")
    return href_text


### general flow (incremental daily latest version crawl)

In [176]:
# general flow (incremental daily latest version crawl)

# create connections to dbs
conn_cve=cve_create_conn()
conn_udd=udd_create_conn()
conn_sqlite=sqlite_create_conn('database.db')

# get all debian binary names (from scantist_library) # example: [(<binary-name>,<description>),...]
# debian_binary_name_list=querycve_all_debian_binary(conn_cve)
debian_binary_name_list = [('cmake',''),('qtbase5-dev','')]

# get all debian binary versions (from scantist_library_version) # example: {(<binary-name>,<description>):[(<version>,]}
binary_version_mapping = querycve_binary_version_date_mapping(conn_cve,debian_binary_name_list)

# # get missing versions for each binary # example: {<binary-name>:{<version1>,<version2>,...},...}
debian_binaries_to_crawl = queryudd_cvetriage_missing_binary_versions(conn_udd,binary_version_mapping)

ftp_list=web_parse_ftp_binary_pool_latest_versions('http://ftp.debian.org/debian/pool/main/c/cmake/')
ftp_dict=web_parse_ftp_binary_pool_latest_versions_map(ftp_list)

querysqlite_insert_library_multiple(conn_sqlite,debian_binaries_to_crawl,ftp_dict)
querysqlite_insert_library_versions_ea_binary(conn_sqlite,'cmake',debian_binaries_to_crawl,ftp_dict)

conn_cve.close()
conn_udd.close()
conn_sqlite.close()

Library_id for cmake is 12


### general flow (full missing version crawl)

In [230]:
# general flow (full missing version crawl)

# create connections to dbs
conn_cve=cve_create_conn()
conn_udd=udd_create_conn()
conn_sqlite=sqlite_create_conn('database.db')

# get all debian binary names (from scantist_library) # example: [(<binary-name>,<description>),...]
# debian_binary_name_list=querycve_all_debian_binary(conn_cve)
# debian_binary_name_list = [('cmake',''),('qtbase5-dev','')]
debian_binary_name_list = [('cmake','')]

# get all debian binary versions (from scantist_library_version) # example: {(<binary-name>,<description>):[(<version>,]}
binary_version_mapping = querycve_binary_version_date_mapping(conn_cve,debian_binary_name_list)

for binary_info in binary_version_mapping.keys():
    print(f'processing {binary_info[0]}')
    df1=web_binary_all_versions_df(binary_info[0])
    diff= set(df1.binary_v.to_list()) - set([_[0] for _ in binary_version_mapping[binary_info]])
    source_name=get_binary_source(binary_info[0])
    print(f'source for {binary_info[0]} is {source_name}')
    df2=queryudd_source_all_version_date_df(conn_udd,source_name)
    df2['date'] = df2['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    merge_df=df1.merge(df2,left_on='source_v',right_on='version',how='outer',indicator=True)
    merge_df[merge_df['_merge']=='both'][['binary_v','date']].to_records(index=False)
    new_versions_to_add = list(merge_df[merge_df['_merge']=='both'][['binary_v','date']].to_records(index=False))
    querysqlite_insert_library_versions_ea_binary2(conn_sqlite,binary_info[0],new_versions_to_add)
    versions_no_date=merge_df[merge_df['_merge']=='left_only'].binary_v.to_list()
    print(f'Added {len(new_versions_to_add)}')
    print(f'Could not obtain date for the following versions [{len(versions_no_date)}]:')
    for _ in merge_df[merge_df['_merge']=='left_only'].binary_v.to_list():
        print(_)

conn_cve.close()
conn_udd.close()
conn_sqlite.close()

processing cmake
source for cmake is cmake


/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_2249/2330383670.py:233: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection_udd)


Library_id for cmake is 12
Could not obtain date for the following versions [53]:
3.27.4-1~bpo12+1
3.27.2-1~bpo12+1
3.26.4-4~bpo12+1
3.25.1-1~bpo11+1
3.25.0-3~bpo11+1
3.24.3-1~bpo11+1
3.24.2-1~bpo11+1
3.24.1-1~bpo11+1
3.24.0-1~bpo11+1
3.23.3-1~bpo11+1
3.23.2-1~bpo11+1
3.23.1-2~bpo11+1
3.23.1-1~bpo11+1
3.23.0-1~bpo11+1
3.22.1-1~bpo11+1
3.22.0-1~bpo11+1
3.21.4-1~bpo11+1
3.21.3-5~bpo11+1
3.21.3-4~bpo11+1
3.18.4-2+deb11u1
3.18.4-2+deb11u1~bpo10+1
3.18.4-2~bpo10+1
3.16.3-3~bpo10+1
3.16.3-1~bpo9+1
3.13.2-1~bpo9+1
3.13.1-1.is.3.10.2-1
3.12.3-3.is.3.10.2-1
3.12.1-1.is.3.10.2-1+qemu
3.12.1-1.1+0.riscv64.2
3.12.1-1+0.riscv64.1
3.11.2-1.is.3.10.2-1+qemu1
3.11.2-1.is.3.10.2-1+qemu
3.11.1-1.is.3.10.2-1+qemu
3.10.2-1~0.riscv64.2
3.10.2-1~0.riscv64.1
3.9.5-1~bpo9+1
3.6.2-2~bpo8+1
3.2.2-2+x32.2
3.2.2-2+x32.1
3.2.2-2+alpha
3.0.2-1+deb8u1
2.8.12.1-1+powerpcspe1
2.8.11.1-1~bpo70+1
2.8.9-1~bpo60+2
2.8.9-1~bpo60+1
2.8.7-4~bpo60+1
2.8.5-1~bpo60+1
2.8.2+dfsg.1-0+squeeze1
2.8.1-5~powerpcspe2
2.8.1-5~powerpcsp

## Archive

## Query Debian binaries from CVETRIAGE

In [1]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../pipeline_monitoring/.env')

# Replace these with your own database credentials
host2 = os.environ.get('CVETRIAGE_POSTGRES_HOSTNAME')
database2 = os.environ.get('CVETRIAGE_POSTGRES_DB')
user2 = os.environ.get('CVETRIAGE_POSTGRES_USERNAME')
password2 = os.environ.get('CVETRIAGE_POSTGRES_PASSWORD')

# Establish the connection
connection_cve = psycopg2.connect(
    host=host2,
    database=database2,
    user=user2,
    password=password2
)

In [3]:
query_debian_select_binaries = f'''SELECT name
from scantist_library
where platform='Debian'
and is_valid=True
'''
df_debian_binaries = pd.read_sql_query(query_debian_select_binaries, connection_cve)
debian_binary_list = df_debian_binaries.name.to_list()
# connection_cve.close()

/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_2249/3815230825.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_debian_binaries = pd.read_sql_query(query_debian_select_binaries, connection_cve)


In [23]:
# for testing purpose
debian_binary_list = ['debhelper','qtbase5-dev','qtbase5-private-dev','deepin-gettext-tools','libdtkwidget-dev','libdtkcore-dev','libdtkgui-dev','libdtkcore5-bin','pkg-config','qttools5-dev-tools','cmake']

binary_version_mapping = {}
cursor = connection_cve.cursor()
for binary_name in debian_binary_list:
    # Execute your SQL query
    query_binary_versions = f'''SELECT slv.version_number, slv.release_date from scantist_library sl
    join scantist_library_version slv on slv.library_id=sl.id
    where sl.name='{binary_name}'
    and sl.platform='Debian'
    and slv.is_valid=True
    order by slv.release_date desc
    '''
    cursor.execute(query_binary_versions)

    # Fetch all the rows as a list of tuples
    result = cursor.fetchall()
    binary_version_mapping[binary_name] = result
cursor.close()
# connection_cve.close()


[('13.9.1~bpo11+1', datetime.datetime(2022, 9, 22, 11, 47, 37, tzinfo=datetime.timezone.utc)), ('13.6', datetime.datetime(2021, 12, 20, 4, 55, 3, tzinfo=datetime.timezone.utc)), ('13.5.2', datetime.datetime(2021, 9, 23, 8, 44, 25, tzinfo=datetime.timezone.utc)), ('13.5.1', datetime.datetime(2021, 8, 30, 21, 43, 35, tzinfo=datetime.timezone.utc)), ('13.5', datetime.datetime(2021, 8, 30, 11, 50, 42, tzinfo=datetime.timezone.utc)), ('13.4.1', datetime.datetime(2021, 8, 21, 1, 1, tzinfo=datetime.timezone.utc)), ('13.4', datetime.datetime(2021, 8, 17, 8, 32, 34, tzinfo=datetime.timezone.utc)), ('13.4+nmu1', datetime.datetime(2021, 8, 17, 5, 34, 2, tzinfo=datetime.timezone.utc)), ('13.3.4', datetime.datetime(2021, 3, 6, 1, 53, 10, tzinfo=datetime.timezone.utc)), ('13.3.3', datetime.datetime(2021, 2, 3, 11, 43, 32, tzinfo=datetime.timezone.utc)), ('13.3.2', datetime.datetime(2021, 1, 31, 8, 53, 24, tzinfo=datetime.timezone.utc)), ('13.3.1', datetime.datetime(2020, 12, 28, 0, 52, 1, tzinfo=dat

## read from UDD

In [133]:
def udd_create_conn():
    host1 = "udd-mirror.debian.net"
    database1 = "udd"
    user1 = "udd-mirror"
    password1 = "udd-mirror"

    # Establish the connection
    connection_udd = psycopg2.connect(
        host=host1,
        database=database1,
        user=user1,
        password=password1
    )
    return connection_udd

In [29]:
cursor = connection_udd.cursor()
debian_binaries_to_crawl = {}
for binary_name in binary_version_mapping.keys():
    # Execute your SQL query
    query_latest_binary_versions = f"SELECT distinct version FROM public.packages where package='{binary_name}'"
    cursor.execute(query_latest_binary_versions)
    # Fetch all the rows as a list of tuples
    udd_latest_versions_tup_list = cursor.fetchall()
    udd_latest_versions_set = set([tup[0] for tup in udd_latest_versions_tup_list])
    cve_versions_set = set(tup[0] for tup in binary_version_mapping[binary_name])
    missing_latest_versions_set = udd_latest_versions_set - cve_versions_set
    debian_binaries_to_crawl[binary_name]=missing_latest_versions_set
cursor.close()

## webcrawl

In [40]:
import requests
from bs4 import BeautifulSoup

def parse_ftp_binary_pool_latest_versions(url):
    file_info_list = []
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the webpage using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <tr> elements on the webpage
        tr_elements = soup.find_all('tr')

        # Extract the names and last modified timestamps from each <tr> element
        
        for tr in tr_elements:
            td_elements = tr.find_all('td')
            if len(td_elements) >= 4:
                name = td_elements[1].text.strip()
                last_modified = td_elements[2].text.strip()
                file_info_list.append((name, last_modified))
        file_info_list.remove(('Parent Directory', ''))
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    
    return file_info_list

In [49]:
ftp_list=parse_ftp_binary_pool_latest_versions('http://ftp.debian.org/debian/pool/main/c/cmake/')

In [120]:
ftp_list

[('cmake-curses-gui_3.13.4-1_amd64.deb', '2019-02-10 12:02'),
 ('cmake-curses-gui_3.13.4-1_arm64.deb', '2019-02-10 11:57'),
 ('cmake-curses-gui_3.13.4-1_armel.deb', '2019-02-10 15:01'),
 ('cmake-curses-gui_3.13.4-1_armhf.deb', '2019-02-10 15:31'),
 ('cmake-curses-gui_3.13.4-1_i386.deb', '2019-02-10 11:26'),
 ('cmake-curses-gui_3.13.4-1_mips.deb', '2019-02-10 16:17'),
 ('cmake-curses-gui_3.13.4-1_mips64el.deb', '2019-02-10 15:31'),
 ('cmake-curses-gui_3.13.4-1_mipsel.deb', '2019-02-10 14:30'),
 ('cmake-curses-gui_3.13.4-1_ppc64el.deb', '2019-02-10 11:26'),
 ('cmake-curses-gui_3.13.4-1_s390x.deb', '2019-02-10 11:31'),
 ('cmake-curses-gui_3.18.4-2+deb11u1_amd64.deb', '2021-11-06 21:12'),
 ('cmake-curses-gui_3.18.4-2+deb11u1_arm64.deb', '2021-11-06 21:27'),
 ('cmake-curses-gui_3.18.4-2+deb11u1_armel.deb', '2021-11-06 21:57'),
 ('cmake-curses-gui_3.18.4-2+deb11u1_armhf.deb', '2021-11-06 21:17'),
 ('cmake-curses-gui_3.18.4-2+deb11u1_i386.deb', '2021-11-06 21:07'),
 ('cmake-curses-gui_3.18.4-

In [85]:
ftp_dict={}
for _ in ftp_list:
    if _[0].endswith('.deb'): # filter out non deb files
        binary_name=_[0].split('_')[0]
        binary_version=_[0].split('_')[1]
        
        release_date=_[1].split(' ')[0]
        if binary_name not in ftp_dict:
            ftp_dict[binary_name]={}
        if binary_version in ftp_dict[binary_name]:
            ftp_dict[binary_name][binary_version].update([release_date])
        else:
            ftp_dict[binary_name][binary_version]=set([release_date])

## temp db for testing

In [102]:
import sqlite3

# Create or connect to an SQLite database file (e.g., "mydatabase.db")
conn = sqlite3.connect("database.db")

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute('''CREATE TABLE IF NOT EXISTS scantist_library (
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	created timestamptz NOT NULL,
	modified timestamptz NOT NULL,
	"name" varchar(255) NULL,
	description text NULL,
	vendor varchar(255) NULL,
	"language" varchar(64) NULL,
	platform varchar(255) NULL,
	"source" varchar(255) NULL,
	processed_time timestamptz NULL,
	is_valid bool NULL,
	CONSTRAINT scantist_library_name_vendor_platform_source_db9dacce_uniq UNIQUE (name, vendor, platform)
);''')

cursor.execute('''CREATE TABLE IF NOT EXISTS scantist_library_version (
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	created timestamptz NOT NULL,
	modified timestamptz NOT NULL,
	version_number varchar(255) NULL,
	release_date timestamptz NULL,
	is_active bool NOT NULL,
	is_officially_supported bool NOT NULL,
	library_id int4 NULL,
	license_id int4 NULL,
	is_clean bool NOT NULL,
	"source" varchar(16) NULL,
	processed_time timestamptz NULL,
	is_valid bool NULL,
	CONSTRAINT scantist_library_version_unique_key UNIQUE (library_id, version_number)
);''')

# Commit the changes to save them to the database file
conn.commit()

# Close the cursor and the connection
cursor.close()
conn.close()


In [118]:
from datetime import datetime

def insert_latest_version(binary_name):
    debian_binary_tup=[]
    curr_time=datetime.now().strftime("%Y-%m-%d")
    for v in debian_binaries_to_crawl[binary_name]:
        if v in ftp_dict[binary_name].keys():
            debian_binary_tup.append((curr_time,curr_time,binary_name,'','','','Debian','Scantist',None,True))
        
    columns_library=', '.join(['created','modified','name','description','vendor','language','platform','source','processed_time','is_valid'])
    columns_library_version=', '.join(['created','modified','version_number','release_date','is_active','is_officially_supported','library_id','license_id','is_clean','source','processed_time','is_valid'])
    # Insert the values into the table using executemany
    sql_query_insert_library = f"INSERT OR IGNORE INTO scantist_library ({columns_library}) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    sql_query_insert_library_version = f"INSERT OR IGNORE INTO scantist_library_version ({columns_library_version}) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    sql_query_select_library_id=f"select id from scantist_library where name='{binary_name}'"
    # Create or connect to an SQLite database file (e.g., "mydatabase.db")

    conn = sqlite3.connect("database.db")
    # Create a cursor object
    cursor = conn.cursor()
    cursor.executemany(sql_query_insert_library, debian_binary_tup)
    # Commit the changes to save them to the database file
    conn.commit()

    cursor.execute(sql_query_select_library_id)
    result_id = cursor.fetchall()[0][0]
    print(f'library_id for {binary_name} is {result_id}')

    debian_binary_v_tup=[]
    for v in debian_binaries_to_crawl[binary_name]:
        if v in ftp_dict[binary_name].keys():
            debian_binary_v_tup.append((curr_time,curr_time,v,min(ftp_dict[binary_name][v]),True,True,result_id,None,True,'Scantist',None,True))

    cursor.executemany(sql_query_insert_library_version, debian_binary_v_tup)
    # Commit the changes to save them to the database file
    conn.commit()

    # Close the cursor and the connection
    cursor.close()
    conn.close()


In [119]:
insert_latest_version('cmake')

library_id for cmake is 12


In [134]:
import requests
from bs4 import BeautifulSoup
import re

def binary_all_versions_df(binary_name):
    url = f'https://snapshot.debian.org/binary/{binary_name}/'
    response = requests.get(url)

    # Initialize an empty list to store the extracted text
    text_list = []

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all the <ul> elements
        ul_elements = soup.find_all('ul')

        # Loop through the <ul> elements to find <li> and <a> elements
        for ul in ul_elements:
            # Find all the <li> elements within the current <ul>
            li_elements = ul.find_all('li')
            
            # Loop through the <li> elements to find <a> elements
            for li in li_elements:
                # Find all the <a> elements within the current <li>
                a_elements = li.find_all('a')
                
                # Loop through the <a> elements to extract and append their text to the list
                for a in a_elements:
                    text = a.get_text()
                    text_list.append(text)
    binary_pkg_v_map={}
    for text in text_list:
        # Define a regular expression pattern to match 'XXX' and 'ZZZ@WWW'
        # pattern = r'/(\b[^()]+\b)\s\(source:\s*[^()]+\s([^()]+)\)/gm'
        pattern = r'(\S+)\s\(source:\s*[^()]+\s(\S+)\)'

        # Use re.search to find the pattern in the text
        match = re.search(pattern, text)

        # Check if a match is found
        if match:
            binary_v = match.group(1)  
            pkg_v = match.group(2)  
            binary_pkg_v_map[binary_v]=pkg_v

    df1 = pd.DataFrame(binary_pkg_v_map.items(), columns=['binary_v', 'package_v'])
    return df1

def udd_source_all_version_date(source_name):
    connection_udd= udd_create_conn()
    query = f"SELECT version, date FROM upload_history where source='{source_name}'"
    df = pd.read_sql_query(query, connection_udd)
    connection_udd.close()
    return df

In [135]:
df_A=binary_all_versions_df('cmake')
df_B=udd_source_all_version_date('cmake')
merged_df = df_A.merge(df_B, left_on='package_v', right_on='version', how='outer', indicator=True)
version_date_map = list(merged_df[merged_df['_merge']=='both'][['version','date']].to_records(index=False))

/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_2249/550370451.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection_udd)


[('3.27.5-1', Timestamp('2023-09-15 12:20:54+0000', tz='UTC')),
 ('3.27.4-1', Timestamp('2023-08-26 18:20:56+0000', tz='UTC')),
 ('3.27.3-1', Timestamp('2023-08-21 09:07:50+0000', tz='UTC')),
 ('3.27.2-1', Timestamp('2023-08-11 10:50:02+0000', tz='UTC')),
 ('3.27.1-2', Timestamp('2023-07-29 22:21:08+0000', tz='UTC')),
 ('3.27.1-2', Timestamp('2023-07-29 22:21:08+0000', tz='UTC')),
 ('3.27.1-1', Timestamp('2023-07-26 09:09:36+0000', tz='UTC')),
 ('3.27.0-2', Timestamp('2023-07-25 15:40:11+0000', tz='UTC')),
 ('3.27.0-1', Timestamp('2023-07-19 23:36:01+0000', tz='UTC')),
 ('3.27.0~rc5-1', Timestamp('2023-07-14 18:29:32+0000', tz='UTC')),
 ('3.27.0~rc4-3', Timestamp('2023-07-14 14:42:14+0000', tz='UTC')),
 ('3.27.0~rc4-2', Timestamp('2023-07-12 13:20:59+0000', tz='UTC')),
 ('3.27.0~rc4-1', Timestamp('2023-07-07 11:08:02+0000', tz='UTC')),
 ('3.26.4-4', Timestamp('2023-06-15 21:52:45+0000', tz='UTC')),
 ('3.26.4-3', Timestamp('2023-06-02 21:20:51+0000', tz='UTC')),
 ('3.26.4-2', Timestamp(

In [129]:
df_A

,binary_v,package_v
0,3.27.5-1,3.27.5-1
1,3.27.4-1,3.27.4-1
2,3.27.4-1~bpo12+1,3.27.4-1~bpo12+1
3,3.27.3-1,3.27.3-1
4,3.27.2-1,3.27.2-1
...,...,...
260,2.0.6-2,2.0.6-2
261,2.0.6-1,2.0.6-1
262,2.0.5-1,2.0.5-1
263,2.0.3-1,2.0.3-1


In [42]:
df1

,binary_v,package_v
0,3.27.5-1,3.27.5-1
1,3.27.4-1,3.27.4-1
2,3.27.4-1~bpo12+1,3.27.4-1~bpo12+1
3,3.27.3-1,3.27.3-1
4,3.27.2-1,3.27.2-1
...,...,...
260,2.0.6-2,2.0.6-2
261,2.0.6-1,2.0.6-1
262,2.0.5-1,2.0.5-1
263,2.0.3-1,2.0.3-1


In [43]:
query = "SELECT version, date FROM upload_history where source='cmake'"
df2 = pd.read_sql_query(query, connection_udd)
# connection_udd.close()

/var/folders/hs/yzhw94b12617bzv5gygdhc780000gn/T/ipykernel_13958/1965905114.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query, connection_udd)


In [44]:
df2

,version,date
0,0.96-1,2001-12-04 19:57:16+00:00
1,0.98-1,2001-12-19 19:56:35+00:00
2,1.0-1,2002-01-11 20:01:00+00:00
3,1.2-1,2002-01-17 19:55:23+00:00
4,1.2-2,2002-02-15 19:57:31+00:00
...,...,...
190,3.27.1-2,2023-07-29 22:21:08+00:00
191,3.27.2-1,2023-08-11 10:50:02+00:00
192,3.27.3-1,2023-08-21 09:07:50+00:00
193,3.27.4-1,2023-08-26 18:20:56+00:00


In [46]:
merged_df = df1.merge(df2, left_on='package_v', right_on='version', how='outer', indicator=True)  # Use 'how' parameter to specify the type of join (inner, outer, left, right)

In [52]:
merged_df[merged_df['_merge']=='both']

,binary_v,package_v,version,date,_merge
0,3.27.5-1,3.27.5-1,3.27.5-1,2023-09-15 12:20:54+00:00,both
1,3.27.4-1,3.27.4-1,3.27.4-1,2023-08-26 18:20:56+00:00,both
2,3.27.4-1~bpo12+1,3.27.4-1~bpo12+1,NaN,NaT,left_only
3,3.27.3-1,3.27.3-1,3.27.3-1,2023-08-21 09:07:50+00:00,both
4,3.27.2-1,3.27.2-1,3.27.2-1,2023-08-11 10:50:02+00:00,both
...,...,...,...,...,...
275,NaN,NaN,1.4.7-1,2003-01-07 00:47:12+00:00,right_only
276,NaN,NaN,1.6.4-1,2003-02-15 17:17:35+00:00,right_only
277,NaN,NaN,1.6.5-1,2003-02-24 17:47:29+00:00,right_only
278,NaN,NaN,1.8.2.1-1,2003-12-06 23:02:07+00:00,right_only


## verify with cvetriage

In [81]:
merged_df2[merged_df2['_merge']=='right_only']

,binary_v,package_v,version_number,release_date,_merge
265,NaN,NaN,2.0.2-1,2004-06-22 21:51:38+00:00,right_only
266,NaN,NaN,2.0.1-1,2004-06-07 06:08:50+00:00,right_only
267,NaN,NaN,1.8.2.1-1,2003-11-26 05:46:54+00:00,right_only
268,NaN,NaN,1.8.1-1,2003-11-08 17:43:02+00:00,right_only
269,NaN,NaN,1.6.7-2,2003-06-22 21:39:03+00:00,right_only
270,NaN,NaN,1.6.7-1,2003-06-22 21:06:46+00:00,right_only
271,NaN,NaN,1.6.5-1,2003-02-23 19:25:58+00:00,right_only
272,NaN,NaN,1.6.4-1,2003-02-14 00:06:12+00:00,right_only
273,NaN,NaN,1.4.7-1,2003-01-06 03:02:36+00:00,right_only
274,NaN,NaN,1.4.6-1,2002-10-24 05:19:47+00:00,right_only
